### 3.3.1. 生成数据集

In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

### 3.3.2. 读取数据集

In [2]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

### 3.3.3. 定义模型

In [3]:
# nn是神经网络的缩写
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

### 3.3.4. 初始化模型参数

In [4]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

### 3.3.5. 定义损失函数

In [5]:
""" loss = nn.MSELoss() """
""" loss = nn.MSELoss(reduction='sum') """
loss = nn.HuberLoss()

### 3.3.6. 定义优化算法

In [6]:
""" trainer = torch.optim.SGD(net.parameters(), lr=0.03) """
trainer = torch.optim.SGD(net.parameters(), lr=0.03) # if nn.MSELoss(reduction='sum')

### 3.3.7 训练

In [7]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        for param in net.parameters():
            print(param.grad)
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

tensor([[-0.0186,  0.1715]])
tensor([-0.8419])
tensor([[0.2828, 0.6352]])
tensor([-0.8000])
tensor([[-0.1669,  0.3447]])
tensor([-0.8754])
tensor([[-0.6304,  0.7990]])
tensor([-0.8000])
tensor([[-0.5429, -0.1079]])
tensor([-0.8560])
tensor([[-0.3561, -0.1918]])
tensor([-0.9383])
tensor([[-0.3864,  0.1793]])
tensor([-0.4000])
tensor([[-0.4370,  0.3201]])
tensor([-0.8454])
tensor([[-0.4337,  0.3788]])
tensor([-0.6817])
tensor([[-0.3304,  0.1547]])
tensor([-0.8712])
tensor([[-0.0869, -0.0338]])
tensor([-0.8818])
tensor([[-0.3747,  0.1630]])
tensor([-0.8873])
tensor([[-0.3690,  0.5354]])
tensor([-0.6075])
tensor([[-0.5352,  0.1904]])
tensor([-0.6271])
tensor([[-0.5364,  0.5515]])
tensor([-0.5474])
tensor([[-0.2162,  0.1285]])
tensor([-1.0000])
tensor([[-0.1239,  0.0990]])
tensor([-0.6833])
tensor([[-0.7789,  0.4829]])
tensor([-0.4000])
tensor([[-0.5889,  0.4518]])
tensor([-0.9220])
tensor([[-0.2260, -0.0129]])
tensor([-0.9729])
tensor([[-0.2294,  0.7945]])
tensor([-0.5581])
tensor([[-0.197

### Practice
#### 问题解析和解答

---

##### **1. 如果将小批量的总损失替换为小批量损失的平均值，需要如何更改学习率？**

**答：**

将小批量的总损失替换为小批量损失的平均值时，梯度的大小会被缩小到原来的 $1 / \text{batch\_size}$。为了保持优化过程的稳定性，学习率通常需要相应地调整。

- 如果框架没有自动归一化处理梯度，可以 **增大学习率**，通常将学习率乘以 $\text{batch\_size}$。
- 现代深度学习框架（如 TensorFlow 和 PyTorch）通常已经对梯度进行了归一化，因此无需手动调整学习率。

---

##### **2. 深度学习框架中提供了哪些损失函数和初始化方法？**

**答：**

###### **常见损失函数：**

- **回归任务**：
  - 均方误差损失（Mean Squared Error, MSE）
  - 平均绝对误差损失（Mean Absolute Error, MAE）
  - Huber 损失（Huber Loss）
  - 对数余弦损失（Log-Cosh Loss）
  
- **分类任务**：
  - 交叉熵损失（Cross-Entropy Loss）
  - 二分类交叉熵损失（Binary Cross-Entropy Loss）
  - Kullback-Leibler 散度（KL Divergence）

- **其他**：
  - 自定义损失函数

###### **常见初始化方法：**

- 随机初始化（Random Initialization）
- **标准初始化方法**：
  - Xavier 初始化（Glorot Initialization）
  - He 初始化（Kaiming Initialization）
  - 均匀分布初始化（Uniform Distribution）
  - 正态分布初始化（Normal Distribution）
- 零初始化和常数初始化（Zero and Constant Initialization）

###### **Huber 损失公式：**

$$
l(y, y') =
\begin{cases} 
|y - y'| - \frac{\sigma}{2}, & \text{if } |y - y'| > \sigma \\ 
\frac{1}{2\sigma}(y - y')^2, & \text{otherwise}.
\end{cases}
$$

在实现时，需要设定超参数 $\sigma$，用以控制线性和二次损失之间的切换点。框架通常会提供默认值，同时允许用户自定义。

---

##### **3. 如何访问线性回归的梯度？**

**答：**

在深度学习框架中，线性回归的梯度可以通过自动微分工具（如 TensorFlow 的 `tf.GradientTape` 或 PyTorch 的 `autograd`）进行计算。

##### 示例代码如下：

##### **PyTorch 示例：**

```python
import torch
import torch.nn as nn

# 定义模型
model = nn.Linear(1, 1)
criterion = nn.MSELoss()

# 输入和标签
x = torch.tensor([[1.0], [2.0]], requires_grad=True)
y = torch.tensor([[2.0], [4.0]])

# 前向传播
output = model(x)
loss = criterion(output, y)

# 反向传播
loss.backward()

# 访问梯度
for param in model.parameters():
    print(param.grad)